## Introduction

In this lab assessment, you'll practice your knowledge of JOIN statements and subqueries, using various types of joins and various methods for specifying the links between them. One of the main benefits of using a relational database is the table relations that define them which allow you to access and connect data together via shared columns. By writing more advanced SQL queries that utilize joins and subqueries you can provide a deeper and more granular level of analysis and data retrieval.

This assessment will continue looking at the familiar Northwind database that contains customer relationship management (CRM) data as well as employee and product data. You will take a deeper dive into this database in order to accomplish more advanced SQL queries that require you to access data from multiple tables at once. 

Imagine that you are working in an analyst role for the sales rep team. They have collaborated with the customer relations and the product teams to take a comprehensive look at the employee to customer pipeline in an attempt to find areas of improvement and potential growth. You have been asked to provide some specific data and statistics regarding this project.

## Learning Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Choose and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Database

The database will be the customer relationship management (CRM) database, which has the following ERD.

![Database-Schema.png](ERD.png)

### Connect to the database

In the cell below we have provided the code to import both pandas and sqlite3 as well as define and create the connection to the database you will use. Also displayed is the schema and table names from the database. Use this information in conjunction with the ERD image above to assist in creating your SQL Queries.

Major Hint: Look for the shared columns across tables you need to 'join' together.

In [2]:
# CodeGrade step0
# Run this cell without changes

# SQL Library and Pandas Library
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.sqlite')

pd.read_sql("""SELECT * FROM sqlite_master""", conn)

,type,name,tbl_name,rootpage,sql
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Part 1: Join and Filter

### Step 1

The company would like to let Boston employees go remote but need to know more information about who is working in that office. Return the first and last names and the job titles for all employees in Boston.

In [3]:
# CodeGrade step1
# Replace None with your code
df_boston = pd.read_sql("""
SELECT firstName, lastName, jobTitle
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE o.city = 'Boston';
""", conn)

### Step 2

Recent downsizing and employee attrition have caused some mixups in office tracking and the company is worried they are supporting a 'ghost' location. Are there any offices that have zero employees?

In [4]:
# CodeGrade step2
# Replace None with your code
df_zero_emp = pd.read_sql("""
SELECT o.officeCode, o.city
FROM offices o 
LEFT JOIN employees e ON e.officeCode = o.officeCode
WHERE e.employeeNumber IS NULL;
""", conn)

## Part 2: Type of Join

### Step 3

As a part of this larger analysis project the HR department is taking the time to audit employee records to make sure nothing is out of place and have asked you to produce a report of all employees. Return the employees first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [5]:
# CodeGrade step3
# Replace None with your code
df_employee = pd.read_sql("""
SELECT e.firstName, e.lastName, o.city, o.state
FROM employees e
LEFT JOIN offices o ON e.officeCode = o.officeCode
GROUP BY e.employeeNumber
ORDER BY e.firstName, e.lastName;
""", conn)

### Step 4
The customer management and sales rep team know that they have several 'customers' in the system that have not placed any orders. They want to reach out to these customers with updated product catalogs to try and get them to place initial orders. Return all of the customer's contact information (first name, last name, and phone number) as well as their sales rep's employee number for any customer that has not placed an order. Sort the results alphabetically based on the contact's last name

There are several approaches you could take here, including a left join and filtering on null values or using a subquery to filter out customers who do have orders. In total there are 24 customers who have not placed an order.

In [6]:
# CodeGrade step4
# Replace None with your code
df_contacts = pd.read_sql("""
SELECT c.contactFirstName, c.contactLastName, c.phone, c.salesRepEmployeeNumber
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
WHERE o.customerNumber IS NULL
GROUP BY c.customerNumber
ORDER BY c.contactFirstName ASC;
""", conn)

## Part 3: Built-in Function

### Step 5

The accounting team is auditing their figures and wants to make sure all customer payments are in alignment, they have asked you to produce a report of all the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. They have asked that these results be sorted in descending order by the payment amount.

Hint: A member of their team mentioned that they are not sure the 'amount' column is being stored as the right datatype so keep this in mind when sorting.

In [7]:
# CodeGrade step5
# Replace None with your code
df_payment = pd.read_sql("""
SELECT c.contactFirstName, c.contactLastName, CAST(p.amount AS REAL) AS amount, p.paymentdate
FROM customers c 
INNER JOIN payments p ON c.customerNumber = p.customerNumber
ORDER BY amount DESC;
""", conn)

df_payment


,contactFirstName,contactLastName,amount,paymentDate
0,Diego,Freyre,120166.58,2005-03-18
1,Diego,Freyre,116208.40,2004-12-31
2,Susan,Nelson,111654.40,2003-08-15
3,Eric,Natividad,105743.00,2003-12-26
4,Susan,Nelson,101244.59,2005-03-05
...,...,...,...,...
268,Carine,Schmitt,1676.14,2004-12-18
269,Pascale,Cartrain,1627.56,2003-04-19
270,Jonas,Bergulfsen,1491.38,2003-10-28
271,Pascale,Cartrain,1128.20,2003-08-22


## Part 4: Joining and Grouping

### Step 6

The sales rep team has noticed several key team members that stand out as having trustworthy business relations with their customers, reflected by high credit limits indicating more potential for orders. The team wants you to identify these 4 individuals. Return the employee number, first name, last name, and number of customers for employees whose customers have an average credit limit over 90k. Sort by number of customers from high to low.

In [28]:
# CodeGrade step6
# Replace None with your code
df_credit = pd.read_sql("""
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(DISTINCT c.customerNumber) AS n_customers, AVG(CAST(c.creditLimit AS REAL)) AS avg_credit_limit
FROM employees e
JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING AVG(CAST(c.creditLimit AS REAL)) > 90000
ORDER BY n_customers DESC, avg_credit_limit DESC
LIMIT 4;
""", conn)

### Step 7

The product team is looking to create new model kits and wants to know which current products are selling the most in order to get an idea of what is popular. Return the product name and count the number of orders for each product as a column named 'numorders'. Also return a new column, 'totalunits', that sums up the total quantity of product sold (use the quantityOrdered column). Sort the results by the totalunits column, highest to lowest, to showcase the top selling products.

In [27]:
# CodeGrade step7
# Replace None with your code
df_product_sold = pd.read_sql("""
SELECT p.productName, COUNT(DISTINCT o.orderNumber) AS numorders, SUM(CAST(o.quantityOrdered AS REAL)) AS totalunits
FROM products p
JOIN orderdetails o ON o.productCode = p.productCode
GROUP BY p.productName
ORDER BY totalunits DESC;
""", conn)

,productName,numorders,totalunits
0,1992 Ferrari 360 Spider red,53,1808.0
1,1937 Lincoln Berline,28,1111.0
2,American Airlines: MD-11S,28,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076.0
4,1930 Buick Marquette Phaeton,28,1074.0
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855.0
105,1911 Ford Town Car,25,832.0
106,1936 Mercedes Benz 500k Roadster,25,824.0
107,1970 Chevy Chevelle SS 454,25,803.0


## Part 5: Multiple Joins

### Step 8

As a follow-up to the above question, the product team also wants to know how many different customers ordered each product to get an idea of market reach. Return the product name, code, and the total number of customers who have ordered each product, aliased as 'numpurchasers'. Sort the results by the highest  number of purchasers.

Hint: You might need to join more than 2 tables. Use DISTINCT to return unique/different values.

In [32]:
# CodeGrade step8
# Replace None with your code
df_total_customers = pd.read_sql("""
SELECT p.productName, p.productCode, COUNT(DISTINCT c.customerName) AS numpurchasers
FROM products p
JOIN orderdetails d ON p.productCode = d.productCode
JOIN orders o ON o.orderNumber = d.orderNumber
JOIN customers c ON c.customerNumber = o.customerNumber
GROUP BY p.productName, p.productCode
ORDER BY numpurchasers DESC;
""", conn)

### Step 9

The custom relations team is worried they are not staffing locations properly to account for customer volume. They want to know how many customers there are per office. Return the count as a column named 'n_customers'. Also return the office code and city.

In [30]:
# CodeGrade step9
# Replace None with your code
df_customers = pd.read_sql("""
SELECT o.officeCode, o.city, COUNT(DISTINCT c.customerName) AS n_customers
FROM offices o
JOIN employees e ON e.officeCode = o.officeCode
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY o.officeCode, o.city
ORDER BY n_customers DESC;
""", conn)

## Part 6: Subquery

### Step 10

Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

In [37]:
# CodeGrade step10
# Replace None with your code
df_under_20 = pd.read_sql("""
WITH under_20 AS (
    SELECT
        od.productCode,
        COUNT(DISTINCT o.customerNumber) AS cust_cnt
    FROM orderdetails od
    JOIN orders o ON o.orderNumber = od.orderNumber
    GROUP BY od.productCode
    HAVING COUNT(DISTINCT o.customerNumber) < 20
)
SELECT DISTINCT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city,
    o.officeCode
FROM employees e
JOIN offices o ON o.officeCode = e.officeCode
JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders o ON o.customerNumber = c.customerNumber
JOIN orderdetails od ON od.orderNumber = o.orderNumber
JOIN under_20 u ON u.productCode = od.productCode
ORDER BY e.lastName, e.firstName, o.officeCode;
""", conn)

### Close the connection

In [38]:
# Run this cell without changes

conn.close()